<a href="https://colab.research.google.com/github/Rishiatweb/Wildfire-detection/blob/main/Forest_Fire_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##direct import as similar to kaggle code
import kagglehub
read_only_path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")
print("Path to dataset files:", read_only_path)

100%|██████████| 1.45G/1.45G [01:09<00:00, 22.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1


In [3]:
import os
import shutil

# Define the source (read-only) and destination (writable) paths
source_dir = read_only_path
writable_dir = "/content/wildfire_dataset_writable"

# Copy the entire directory tree
print(f"\nCopying dataset from {source_dir} to {writable_dir}...")
if os.path.exists(writable_dir):
    shutil.rmtree(writable_dir) # Remove if it exists to ensure a fresh copy
shutil.copytree(source_dir, writable_dir)
print("Copying complete.")

# --- IMPORTANT: Update path variables to point to the new writable directories ---
train_dir = os.path.join(writable_dir, 'train')
valid_dir = os.path.join(writable_dir, 'valid')
test_dir = os.path.join(writable_dir, 'test')

# Verify the new paths
print("\nUpdated paths to writable directories:")
print(f"Train directory: {train_dir} (Exists: {os.path.exists(train_dir)})")
print(f"Validation directory: {valid_dir} (Exists: {os.path.exists(valid_dir)})")
print(f"Test directory: {test_dir} (Exists: {os.path.exists(test_dir)})")


Copying dataset from /root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1 to /content/wildfire_dataset_writable...
Copying complete.

Updated paths to writable directories:
Train directory: /content/wildfire_dataset_writable/train (Exists: True)
Validation directory: /content/wildfire_dataset_writable/valid (Exists: True)
Test directory: /content/wildfire_dataset_writable/test (Exists: True)


In [4]:
import tensorflow as tf

def find_and_remove_tf_unreadable_images(base_dir):
    corrupt_files = []
    total_files = 0
    print(f"Scanning directory: {base_dir}")
    for root, _, files in os.walk(base_dir):
        for file in files:
            total_files += 1
            file_path = os.path.join(root, file)
            try:
                raw_image = tf.io.read_file(file_path)
                tf.io.decode_jpeg(raw_image)
            except tf.errors.InvalidArgumentError as e:
                print(f"--> Found problematic file for TensorFlow: {file_path}")
                corrupt_files.append(file_path)
            except Exception as e:
                print(f"--> Found other problematic file: {file_path} - Error: {e}")
                corrupt_files.append(file_path)

    if corrupt_files:
        print(f"\nFound {len(corrupt_files)} problematic files. Removing them...")
        for file_path in corrupt_files:
            try:
                os.remove(file_path)
                print(f"Successfully removed: {file_path}")
            except OSError as e:
                print(f"Error removing file {file_path}: {e}")
        print("Removal complete.")
    else:
        print("No problematic files found by TensorFlow decoder.")
    print(f"\nScanned {total_files} files in total in {base_dir}.")

# --- Run the cleaning process on the writable copies ---
print("\n--- Cleaning Writable Train Directory ---")
find_and_remove_tf_unreadable_images(train_dir)
print("\n--- Cleaning Writable Validation Directory ---")
find_and_remove_tf_unreadable_images(valid_dir)
print("\n--- Cleaning Writable Test Directory ---")
find_and_remove_tf_unreadable_images(test_dir)
print("\n\nDataset cleaning finished. Proceeding to load data...")


--- Cleaning Writable Train Directory ---
Scanning directory: /content/wildfire_dataset_writable/train
--> Found problematic file for TensorFlow: /content/wildfire_dataset_writable/train/nowildfire/-114.152378,51.027198.jpg

Found 1 problematic files. Removing them...
Successfully removed: /content/wildfire_dataset_writable/train/nowildfire/-114.152378,51.027198.jpg
Removal complete.

Scanned 30250 files in total in /content/wildfire_dataset_writable/train.

--- Cleaning Writable Validation Directory ---
Scanning directory: /content/wildfire_dataset_writable/valid
No problematic files found by TensorFlow decoder.

Scanned 6300 files in total in /content/wildfire_dataset_writable/valid.

--- Cleaning Writable Test Directory ---
Scanning directory: /content/wildfire_dataset_writable/test
--> Found problematic file for TensorFlow: /content/wildfire_dataset_writable/test/wildfire/-73.15884,46.38819.jpg

Found 1 problematic files. Removing them...
Successfully removed: /content/wildfire_da

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, ConfusionMatrixDisplay

In [7]:
#Constants to play around with
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_dataset = image_dataset_from_directory(
    train_dir, labels='inferred', label_mode='binary', image_size=IMG_SIZE,
    interpolation='nearest', batch_size=BATCH_SIZE, shuffle=True)

validation_dataset = image_dataset_from_directory(
    valid_dir, labels='inferred', label_mode='binary', image_size=IMG_SIZE,
    interpolation='nearest', batch_size=BATCH_SIZE, shuffle=False)

test_dataset = image_dataset_from_directory(
    test_dir, labels='inferred', label_mode='binary', image_size=IMG_SIZE,
    interpolation='nearest', batch_size=BATCH_SIZE, shuffle=False)

class_names = train_dataset.class_names
print("Class Names:", class_names)

Found 30249 files belonging to 2 classes.
Found 6300 files belonging to 2 classes.
Found 6299 files belonging to 2 classes.
Class Names: ['nowildfire', 'wildfire']


In [8]:
data_augmentation = Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
], name="data_augmentation")

In [9]:
custom_cnn = Sequential([
    Input(shape=IMG_SIZE + (3,)),
    tf.keras.layers.Rescaling(1./255),
    data_augmentation,
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
], name="custom_cnn")

custom_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
custom_cnn.summary()

print("\n--- Training Custom CNN ---")
history_cnn = custom_cnn.fit(
    train_dataset,
    epochs=20,
    validation_data=validation_dataset
)

Model: "custom_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)


--- Training Custom CNN ---
Epoch 1/20
946/946 ━━━━━━━━━━━━━━━━━━━━ 71s 68ms/step - accuracy: 0.8748 - loss: 0.2945 - val_accuracy: 0.9387 - val_loss: 0.1599
Epoch 2/20
946/946 ━━━━━━━━━━━━━━━━━━━━ 63s 66ms/step - accuracy: 0.9311 - loss: 0.1912 - val_accuracy: 0.9205 - val_loss: 0.1966
Epoch 3/20
946/946 ━━━━━━━━━━━━━━━━━━━━ 93s 78ms/step - accuracy: 0.9403 - loss: 0.1647 - val_accuracy: 0.9424 - val_loss: 0.1658
Epoch 4/20
946/946 ━━━━━━━━━━━━━━━━━━━━ 71s 75ms/step - accuracy: 0.9464 - loss: 0.1519 - val_accuracy: 0.9403 - val_loss: 0.1646
Epoch 5/20
946/946 ━━━━━━━━━━━━━━━━━━━━ 84s 77ms/step - accuracy: 0.9484 - loss: 0.1458 - val_accuracy: 0.9486 - val_loss: 0.1581
Epoch 6/20
946/946 ━━━━━━━━━━━━━━━━━━━━ 65s 68ms/step - accuracy: 0.9492 - loss: 0.1410 - val_accuracy: 0.9600 - val_loss: 0.1156
Epoch 7/20
946/946 ━━━━━━━━━━━━━━━━━━━━ 83s 70ms/step - accuracy: 0.9510 - loss: 0.1369 - val_accuracy: 0.9489 - val_loss: 0.1527
Epoch 8/20
946/946 ━━━━━━━━━━━━━━━━━━━━ 65s 69ms/step - accur

In [ ]:
# --- Part A: Feature Extraction ---
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False

inputs = Input(shape=IMG_SIZE + (3,))
x = tf.keras.layers.Rescaling(1./255)(inputs)
x = data_augmentation(x)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)
transfer_model = Model(inputs, outputs)

transfer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                       loss='binary_crossentropy', metrics=['accuracy'])

print("\n--- Training Transfer Learning Model (Feature Extraction) ---")
history_transfer = transfer_model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

# --- Part B: Fine-Tuning ---
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

transfer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                       loss='binary_crossentropy', metrics=['accuracy'])

print("\n--- Training Transfer Learning Model (Fine-Tuning) ---")
history_fine_tune = transfer_model.fit(
    train_dataset,
    epochs=history_transfer.epoch[-1] + 10,
    initial_epoch=history_transfer.epoch[-1],
    validation_data=validation_dataset
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

--- Training Transfer Learning Model (Feature Extraction) ---
Epoch 1/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 73s 71ms/step - accuracy: 0.8826 - loss: 0.2736 - val_accuracy: 0.9448 - val_loss: 0.1489
Epoch 2/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 79s 69ms/step - accuracy: 0.9367 - loss: 0.1710 - val_accuracy: 0.9486 - val_loss: 0.1410
Epoch 3/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 65s 69ms/step - accuracy: 0.9383 - loss: 0.1683 - val_accuracy: 0.9489 - val_loss: 0.1421
Epoch 4/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - accuracy: 0.9375 - loss: 0.1639 - val_accuracy: 0.9481 - val_loss: 0.1404
Epoch 5/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - accuracy: 0.9393 - loss: 0.1610 - val_accuracy: 0.9521 - val_loss: 0.1314
Epoch 6/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 67s 70ms/step - accuracy: 0.9389 - loss: 0.1641 - val_accuracy: 0.9460 - val_loss: 0.1450
Epoch 7/10
946/946 ━━━━━━━━━━━━━━━━━━━━ 66s 69ms/step - accuracy: 0.9387 - loss: 0.1622 - val_accuracy: 0.95

In [ ]:
# --- Model 3: Transfer Learning with ResNet50 ---

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model

print("\n\n--- Setting up ResNet50 Model ---")

# --- Part A: Feature Extraction ---
# 1. Instantiate the base model
# Note: ResNet50 has its own preprocessing function, but for a fair comparison,
# we'll use the same Rescaling(1./255) as the other models.
base_model_resnet = ResNet50(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

# 2. Freeze the base model
base_model_resnet.trainable = False

# 3. Create the new model by adding our layers on top
inputs_resnet = Input(shape=IMG_SIZE + (3,))
# Apply the same augmentation and rescaling for a fair comparison
x_resnet = data_augmentation(inputs_resnet)
x_resnet = tf.keras.applications.resnet.preprocess_input(x_resnet) # Use ResNet's specific preprocessing
x_resnet = base_model_resnet(x_resnet, training=False)
x_resnet = GlobalAveragePooling2D()(x_resnet)
x_resnet = Dropout(0.2)(x_resnet)
outputs_resnet = Dense(1, activation='sigmoid')(x_resnet)

resnet_model = Model(inputs_resnet, outputs_resnet, name="resnet50_transfer_model")

# 4. Compile the model
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

resnet_model.summary()

# 5. Train the model (feature extraction phase)
print("\n--- Training ResNet50 Model (Feature Extraction) ---")
history_resnet = resnet_model.fit(
    train_dataset,
    epochs=10, # Keeping epochs consistent for comparison
    validation_data=validation_dataset,
    verbose=1
)

# --- Part B: Fine-Tuning ---
# 6. Unfreeze some layers of the base model
base_model_resnet.trainable = True

# Let's fine-tune a smaller portion of ResNet as it's a larger model.
# Let's unfreeze from the last convolutional block (e.g., from layer ~143 onwards).
for layer in base_model_resnet.layers[:143]:
    layer.trainable = False

# 7. Re-compile the model with a very low learning rate
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

print("\n--- Training ResNet50 Model (Fine-Tuning) ---")
# 8. Continue training
history_fine_tune_resnet = resnet_model.fit(
    train_dataset,
    epochs=history_resnet.epoch[-1] + 10, # Train for 10 more epochs
    initial_epoch=history_resnet.epoch[-1],
    validation_data=validation_dataset
)

In [ ]:
# Get true labels and predictions for both models
y_true = np.concatenate([y for x, y in test_dataset], axis=0).flatten()
y_pred_cnn_prob = custom_cnn.predict(test_dataset).flatten()
y_pred_cnn = (y_pred_cnn_prob > 0.5).astype(int)
y_pred_transfer_prob = transfer_model.predict(test_dataset).flatten()
y_pred_transfer = (y_pred_transfer_prob > 0.5).astype(int)
y_pred_resnet_prob = resnet_model.predict(test_dataset).flatten()
y_pred_resnet = (y_pred_resnet_prob > 0.5).astype(int)

print("\n--- Custom CNN Classification Report ---")
print(classification_report(y_true, y_pred_cnn, target_names=class_names))

print("\n--- Transfer Learning Model Classification Report ---")
print(classification_report(y_true, y_pred_transfer, target_names=class_names))

print("\n--- ResNet50 Model Classification Report ---")
print(classification_report(y_true, y_pred_resnet, target_names=class_names))

In [ ]:
# Plot Confusion Matrices
fig, axes = plt.subplots(1, 3, figsize=(21, 6))
# Plot Custom CNN
ConfusionMatrixDisplay.from_predictions(y_true, y_pred_cnn, display_labels=class_names, ax=axes[0], cmap=plt.cm.Blues)
axes[0].set_title('Custom CNN Confusion Matrix')
# Plot MobileNetV2
ConfusionMatrixDisplay.from_predictions(y_true, y_pred_transfer, display_labels=class_names, ax=axes[1], cmap=plt.cm.Blues)
axes[1].set_title('MobileNetV2 Model Confusion Matrix')
# Plot ResNet50
ConfusionMatrixDisplay.from_predictions(y_true, y_pred_resnet, display_labels=class_names, ax=axes[2], cmap=plt.cm.Blues)
axes[2].set_title('ResNet50 Model Confusion Matrix')
plt.tight_layout()
plt.show()

# Plot ROC Curves
fpr_cnn, tpr_cnn, _ = roc_curve(y_true, y_pred_cnn_prob)
roc_auc_cnn = auc(fpr_cnn, tpr_cnn)
fpr_transfer, tpr_transfer, _ = roc_curve(y_true, y_pred_transfer_prob)
roc_auc_transfer = auc(fpr_transfer, tpr_transfer)
fpr_resnet, tpr_resnet, _ = roc_curve(y_true, y_pred_resnet_prob)
roc_auc_resnet = auc(fpr_resnet, tpr_resnet)

plt.figure(figsize=(10, 8))
plt.plot(fpr_cnn, tpr_cnn, color='blue', lw=2, label=f'Custom CNN (AUC = {roc_auc_cnn:.4f})')
plt.plot(fpr_transfer, tpr_transfer, color='yellow', lw=2, label=f'MobileNet (AUC = {roc_auc_transfer:.4f})')
plt.plot(fpr_resnet, tpr_resnet, color='red', lw=2, label=f'ResNet50 (AUC = {roc_auc_resnet:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Receiver Operating Characteristic (ROC) Curve Comparison', fontsize=14)
plt.legend(loc="lower right", fontsize=12)
plt.grid(True)
plt.show()